# RUN KERAS MODEL - Ayukngaji 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os
import random
from tensorflow.python.platform import gfile
import librosa

#for removing future warnings
import warnings
warnings.filterwarnings('ignore')

Note: import keras seperti ini.

In [3]:
# from tf.keras.models import Sequential  # Tidak bisa!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten

This was developed using Python 3.6 (Anaconda) and TensorFlow version:

In [4]:
tf.__version__

'1.6.0'

In [7]:
tf.keras.__version__

'2.1.3-tf'

In [17]:
def init(path):
    labels = os.listdir(path)
    index = 0
    for label in labels:
        LABEL_TO_INDEX_MAP[label] = index
        index += 1
        
    global NUM_LABELS
    NUM_LABELS = len(LABEL_TO_INDEX_MAP)
    print(LABEL_TO_INDEX_MAP)

def one_hot_encoding(label):
    encoding = [0] * len(LABEL_TO_INDEX_MAP)
    encoding[LABEL_TO_INDEX_MAP[label]] = 1
    return encoding

def get_mfcc(wave_path, PAD_WIDTH=WIDTH):
    if DEBUG:
        print(wave_path)
    wave, sr = librosa.load(wave_path, mono=True, duration=0.7)
    mfccs = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=HEIGHT)
    mfccs = np.pad(mfccs,((0,0), (0, PAD_WIDTH - len(mfccs[0]))), mode='constant')
    return mfccs
def get_batch(batch_size, path):
    X = []
    Y = []
    random.seed(5896)
    path = os.path.join(path,'*', '*.wav')
    waves = gfile.Glob(path)
    while True:
        random.shuffle(waves)
        for wave_path in waves:
            _,label = os.path.split(os.path.dirname(wave_path))
            X.append(get_mfcc(wave_path))
            Y.append(one_hot_encoding(label))

            if (len(X) == batch_size):
                yield X, Y
                X = []
                Y = []


Import lagi

In [18]:
"""
DAAATASET
"""
DEBUG = False
PATH_TRAIN = os.getcwd()+'/data/train'
PATH_TEST = os.getcwd()+'/data/test'
LOGDIR = 'log/'
TEST_LOGDIR = 'log_test/'
LABEL_TO_INDEX_MAP = {}
WIDTH = 44
HEIGHT = 25
NUM_LABELS = 0
BATCHRUN = 500
EPOCHS = 5
path_model='model.keras'

In [19]:
from tensorflow.python.keras.models import load_model

Panggil model

In [20]:
model3 = load_model(path_model)

Coba test model yang di load

In [31]:
BATCHRUN=10
init(PATH_TEST)
batch = get_batch(BATCHRUN,PATH_TEST)
X, Y = next(batch)

for i in range(len(X)):
    X[i]=X[i].reshape((25*44,))
    
images = np.array(X)

{'ba': 0, 'ju': 1, 'mi': 2, 'nu': 3, 'si': 4, 'ta': 5}


In [32]:
cls_true = np.array(Y)

We then use the restored model to predict the class-numbers for those images.

In [33]:
y_pred = model3.predict(x=images)

In [34]:
y_pred[1]

array([7.0867548e-12, 9.9286991e-01, 5.4162340e-03, 4.1895447e-36,
       1.7138711e-03, 2.2422614e-14], dtype=float32)

Get the class-numbers as integers.

In [35]:
cls_pred = np.argmax(y_pred, axis=1)

In [36]:
cls_pred

array([1, 1, 2, 1, 1, 2, 5, 0, 5, 2], dtype=int64)

In [37]:
result = model3.evaluate(x=np.array(X),
                        y=np.array(Y))

10/10 [==============================]10/10 [==============================] - 0s 552us/step



In [38]:
#Cek akurasi
for name, value in zip(model3.metrics_names, result):
    print(name, value)

loss 4.865939617156982
acc 0.4000000059604645
